In [ ]:
#######################################################################################
# Author: Srijan Verma                                                              #
# School of Pharmacy                                                                #
# Sirimulla Research Group [http://www.sirimullaresearchgroup.com/]                 #
# The University of Texas at El Paso, TX, USA                                       #
# Last modified: 19/12/2019                                                         #
# Copyright (c) 2019 Srijan Verma and Sirimulla Research Group, under MIT license   #
#######################################################################################

In [1]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm_notebook
import pandas as pd
from glob import glob
import time
import os

In [404]:
url = 'http://bioinformatics.charite.de/transformer/index.php?site=biotransformation'
result = requests.get(url)
c = result.content
soup = BeautifulSoup(c)
soup_string = str(soup)

In [405]:
query_name = []
q_str = 'test'
dyn_str = 'CYPs' #Dynamic
s1 = 'results&amp;cname=' #Constant
s2 = '</a>' #Constant

### results&amp;cname=SC6A2_HUMAN&amp;cyp">Sodium-dependent noradrenaline transporter</a>

In [406]:
while(q_str.split('>')[0] != 'SO4A1_HUMAN&amp;cyp"'):
    
    q_str = soup_string[soup_string.find(s1, soup_string.find(dyn_str)) + len(s1):soup_string.find(s2, (soup_string.find(s1, soup_string.find(dyn_str)) + len(s1)))]
    dyn_str = q_str
    query_name.append(q_str)

In [407]:
len(query_name)

154

In [408]:
prot_name = []
q_name = []
for i in tqdm_notebook(range(len(query_name))):
    prot_name.append(query_name[i].split('>')[1])
    q_name.append(query_name[i].split('>')[0][0:-9])

In [409]:
df = pd.DataFrame(columns = ['target_name','query'])
df['target_name'] = prot_name
df['query'] = q_name
df.to_csv('../dataset/transformer/target_query.csv')

In [410]:
len(set(prot_name))

154

In [320]:
for i in range(len(prot_name)):
    prot_name[i] = prot_name[i].replace(' ', '_')
    prot_name[i] = prot_name[i].replace('-', '_')
    prot_name[i] = prot_name[i].replace(',', '')
    prot_name[i] = prot_name[i].replace('(', '')
    prot_name[i] = prot_name[i].replace(')', '')
    prot_name[i] = prot_name[i].replace('/', '')
#     prot_name[i] = prot_name[i].replace("\", '')

In [322]:
# prot_name

In [286]:
# url = 'http://bioinformatics.charite.de/transformer/index.php?site=fullinfo_results&cname=' + q_name[0] + '&cyp'
url = 'http://bioinformatics.charite.de/transformer/index.php?site=fullinfo_results&cname=SO3A1_HUMAN&cyp'
result = requests.get(url)
c = result.content
soup = BeautifulSoup(c)
soup_string = str(soup)

In [287]:
# q_str = 'Drug Interactions'
dyn_str = 'Drug Interactions' #Dynamic
s1 = 'href="index.php?site=fullinfo&amp;cname=' #Constant
s2 = '</a>' #Constant
s3 = '<td>'
s4 = '</td>'
drug_name = ''

In [299]:
df = pd.DataFrame(columns = ['drug_name','drug_type'])
df

,drug_name,drug_type


In [323]:
for i in tqdm_notebook(range(len(q_name))):
    
    url = 'http://bioinformatics.charite.de/transformer/index.php?site=fullinfo_results&cname='+ q_name[i] +'&cyp'
    result = requests.get(url)
    c = result.content
    soup = BeautifulSoup(c)
    soup_string = str(soup)
    
    all_d_name = []
    all_d_type = []
    # q_str = 'Drug Interactions'
    dyn_str = 'Drug Interactions' #Dynamic
    s1 = 'href="index.php?site=fullinfo&amp;cname=' #Constant
    s2 = '</a>' #Constant
    s3 = '<td>'
    s4 = '</td>'
    drug_name = ''
    
    while(soup_string.find(s1, soup_string.find(dyn_str)) != -1):
        drug_name = soup_string[soup_string.find(s1, soup_string.find(dyn_str)) + len(s1):soup_string.find(s2, (soup_string.find(s1, soup_string.find(dyn_str)) + len(s1)))]
        all_d_name.append(drug_name.split('>')[1])
        dyn_str2 = drug_name
        drug_type = soup_string[soup_string.find(s3, soup_string.find(dyn_str2)) + len(s3):soup_string.find(s4, (soup_string.find(s3, soup_string.find(dyn_str2)) + len(s3)))]
        dyn_str = drug_name
        all_d_type.append(drug_type)
        
    df = pd.DataFrame(columns = ['drug_name','drug_type'])
    df['drug_name'] = all_d_name
    df['drug_type'] = all_d_type
    df.to_csv('../dataset/transformer/' + prot_name[i] + '.csv')

In [411]:
from selenium import webdriver

In [426]:
driverpath = '../chrome/chromedriver'
driver = webdriver.Chrome(driverpath)

In [427]:
driver.get('http://bioinformatics.charite.de/transformer/index.php?site=drug_search')

In [353]:
#//*[@id="cname"]

In [354]:
# elem = driver.find_element_by_id("cname")

In [355]:
# elem.send_keys('paracetamol')

In [ ]:
#//*[@id="searchform"]/table/tbody/tr[5]/td/div/input[2]

In [360]:
# driver.find_element_by_xpath('//*[@id="searchform"]/table/tbody/tr[5]/td/div/input[2]').click()

In [368]:
# s1 = 'SMILES </a></td><td>'
# s2 = '<br'
# soup_string = str(driver.page_source)
# smiles = soup_string[soup_string.find(s1) + len(s1):soup_string.find(s2, (soup_string.find(s1) + len(s1)))]

In [414]:
s1 = 'SMILES </a></td><td>'
s2 = '<br'

In [415]:
interesting_files = glob("../dataset/transformer/data_without_smiles/*csv")  
df = pd.concat((pd.read_csv(f, header = 0, index_col=0) for f in tqdm_notebook(interesting_files)))
df_deduplicated = df.drop_duplicates()

In [416]:
df_deduplicated.reset_index(drop=True, inplace=True)
df_deduplicated.drop_duplicates('drug_name', inplace=True)
len(df_deduplicated)

/Users/pankajverma/anaconda3/envs/seqcomhol/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


1031

In [417]:
df_deduplicated.reset_index(drop=True, inplace=True)
df_deduplicated.to_csv('../dataset/transformer/unique_data_without_smiles.csv')

In [431]:
df_deduplicated = pd.read_csv('../dataset/transformer/unique_data_without_smiles.csv', index_col=0)

In [432]:
df_deduplicated.head(2)

,drug_name,drug_type
0,Ciprofibrate,Inducer
1,Clofibrate,"Substrate,Inducer"


In [433]:
smiles = []
error_indices = []
for i in tqdm_notebook(range(len(df_deduplicated['drug_name']))):

    try:
        driver.get('http://bioinformatics.charite.de/transformer/index.php?site=drug_search')
        time.sleep(0.5)
        elem = driver.find_element_by_id("cname")
        time.sleep(0.5)
        elem.send_keys(df_deduplicated['drug_name'][i])
        time.sleep(0.5)
        driver.find_element_by_xpath('//*[@id="searchform"]/table/tbody/tr[5]/td/div/input[2]').click()
        time.sleep(0.5)
        soup_string = str(driver.page_source)
        smiles.append(soup_string[soup_string.find(s1) + len(s1):soup_string.find(s2, (soup_string.find(s1) + len(s1)))])
        
    except:
        smiles.append('-')
        error_indices.append(i)
        pass
    
df_deduplicated['smiles'] = smiles
df_deduplicated.to_csv('../dataset/transformer/data_with_smiles/unique_data_with_smiles.csv')

In [436]:
str(df_deduplicated[df_deduplicated.drug_name == 'Dinoprostone']['smiles'])

'69    IC "-//W3C//DTD HTML 4.01 Transitional//EN" "h...\nName: smiles, dtype: object'

In [440]:
error_no = []
for i in tqdm_notebook(range(len(df_deduplicated))):
    
    if '</div>' in df_deduplicated['smiles'].iloc[i] or 'href' in df_deduplicated['smiles'].iloc[i] or 'src' in df_deduplicated['smiles'].iloc[i] or '<script' in df_deduplicated['smiles'].iloc[i]:
        df_deduplicated['smiles'][i] = '-'
        error_no.append(i)
        continue
        
    else:
        continue

In [442]:
len(error_no)

55

In [443]:
df_deduplicated.to_csv('../dataset/transformer/data_with_smiles/unique_data_with_smiles.csv')

In [2]:
df_deduplicated = pd.read_csv('../dataset/transformer/from_drugbank_and_chembl_and_transformer_cleaned_v2.csv',index_col=0)

In [12]:
df_deduplicated_index = df_deduplicated.set_index('drug_name')
simva_smi = 'CCC(C)(C)C(=O)O[C@H]1C[C@@H](C)C=C2C=C[C@H](C)[C@H](CC[C@@H]3C[C@@H](O)CC(=O)O3)[C@@H]12'
eze_smi = 'O[C@@H](CC[C@H]1[C@H](N(C1=O)C2=CC=C(F)C=C2)C3=CC=C(O)C=C3)C4=CC=C(F)C=C4'
sulfa_smi = 'CC1=CC(NS(=O)(=O)C2=CC=C(N)C=C2)=NO1'
trim_smi = 'COC1=CC(CC2=C(N)N=C(N)N=C2)=CC(OC)=C1OC'

In [4]:
csv_check_again = []
for csv in tqdm_notebook(glob('../dataset/transformer/data_without_smiles/*csv')):
    df = pd.read_csv(csv, index_col=0)
    for j in range(len(df)):
        if df['drug_name'][j] == 'Simvastatin And Ezetimibe' or df['drug_name'][j] == 'Sulfamethoxazole And Trimethoprim':
            csv_check_again.append(csv)

In [27]:
a= 'srijan'
a.split(',')

['srijan']

In [25]:
import os
not_found = []
csv_not_found = []
csv_empty = []
csv_check_again = []
for csv in tqdm_notebook(glob('../dataset/transformer/data_without_smiles/*csv')):
    
    df = pd.read_csv(csv, index_col=0)
    
    smi = []
    dn = []
    dt = []
    sm = []
    
    if len(df) == 0:
        out = os.path.splitext(os.path.basename(csv))[0]
        csv_empty.append(out)
        continue
        
    else:
        for j in range(len(df)):
            
            try:
                smi.append(df_deduplicated_index.loc[df['drug_name'][j]]['smiles'])
            except:
                smi.append('missing')
                csv_not_found.append(csv)
                not_found.append(j)
                pass
            
            if df['drug_name'][j] == 'Simvastatin And Ezetimibe':
                csv_check_again.append(csv)
                split_drug = df['drug_name'][j].split('And')
                dn.append(split_drug[0].strip())
                dn.append(split_drug[1].strip())
                dt.append(df['drug_type'][j])
                dt.append(df['drug_type'][j])
                sm.append(simva_smi)
                sm.append(eze_smi)
            
            elif df['drug_name'][j] == 'Sulfamethoxazole And Trimethoprim':
                csv_check_again.append(csv)
                split_drug = df['drug_name'][j].split('And')
                dn.append(split_drug[0].strip())
                dn.append(split_drug[1].strip())
                dt.append(df['drug_type'][j])
                dt.append(df['drug_type'][j])
                sm.append(sulfa_smi)
                sm.append(trim_smi)
                
            else:
                dn.append(df['drug_name'][j])
                dt.append(df['drug_type'][j])
                sm.append(smi[j])
                
        out = os.path.splitext(os.path.basename(csv))[0]
        
        labels = ['drug_name']
        df_edited = pd.DataFrame(dn, columns=labels)
        df_edited['drug_type'] = dt
        df_edited['Smiles'] = sm

        df_edited.to_csv('../dataset/transformer/data_with_smiles/'+ out +'.csv')


In [31]:
for csv in tqdm_notebook(glob('../dataset/transformer/data_with_smiles/*csv')):
    
    df = pd.read_csv(csv, index_col=0)
    dn = []
    dt = []
    sm = []
    
    for j in range(len(df)):
        
        if ',' in df['drug_type'][j]:
            split_type = df['drug_type'][j].split(',')
            
            for k in range(len(split_type)):
                dn.append(df['drug_name'][j])
                dt.append(split_type[k].strip())
                sm.append(df['Smiles'][j])
            
        else:
            dn.append(df['drug_name'][j])
            dt.append(df['drug_type'][j])
            sm.append(df['Smiles'][j])
        
    out = os.path.splitext(os.path.basename(csv))[0]

    labels = ['drug_name']
    df_edited = pd.DataFrame(dn, columns=labels)
    df_edited['drug_type'] = dt
    df_edited['Smiles'] = sm

    df_edited.to_csv('../dataset/transformer/data_with_smiles/'+ out +'.csv')

In [32]:
a = []
for csv in tqdm_notebook(glob('../dataset/transformer/data_with_smiles/*csv')):
    df = pd.read_csv(csv, index_col=0)
    for j in range(len(df)):
        
#         if ',' in df['drug_type'][j]:
        a.append(df['drug_type'][j])

In [34]:
set(a)

{'Inducer', 'Inhibitor', 'Substrate'}

In [15]:
csv_check_again

['../dataset/transformer/data_without_smiles/2D6.csv',
 '../dataset/transformer/data_without_smiles/2C9.csv',
 '../dataset/transformer/data_without_smiles/2C9.csv',
 '../dataset/transformer/data_without_smiles/2C8.csv',
 '../dataset/transformer/data_without_smiles/3A4.csv',
 '../dataset/transformer/data_without_smiles/S_mephenytoin_4_hydroxylase.csv']

In [460]:
missing_smi_drug_name = []
for i in range(len(df_deduplicated)):
    if df_deduplicated['smiles'][i] == '-':
        missing_smi_drug_name.append(df_deduplicated['drug_name'][i])
        

In [461]:
len(missing_smi_drug_name)

55

In [494]:
get_missing_smi_from_drugBank = []
error_dr = []
for i in tqdm_notebook(range(len(missing_smi_drug_name))):
    
    if missing_smi_drug_name[i] == 'Selenium Compounds' or missing_smi_drug_name[i] == 'Simvastatin And Ezetimibe' or missing_smi_drug_name[i] == 'Thiamine (vit B1)' or missing_smi_drug_name[i] == 'Sulfamethoxazole And Trimethoprim':
        get_missing_smi_from_drugBank.append('check_again')
        continue
        
    else:
        try:
            url = 'https://www.drugbank.ca/unearth/q?utf8=%E2%9C%93&query='+ missing_smi_drug_name[i] + '&searcher=drugs'
            result = requests.get(url)
            c = result.content
            soup = BeautifulSoup(c)
            soup_string = str(soup)
            get_missing_smi_from_drugBank.append(soup_string[soup_string.find('>SMILES</dt>')+61:soup_string.find('</div>', soup_string.find('>SMILES</dt>')+40)])

        except:
            error_dr.append(missing_smi_drug_name[i])
            get_missing_smi_from_drugBank.append('-')
            pass
# str5 = soup_string[soup_string.find('>Protein binding<')+55:soup_string.find('>Metabolism<')-39]


In [496]:
dr_check_again = []
for i in range(len(get_missing_smi_from_drugBank)):
    if 'href' in get_missing_smi_from_drugBank[i] or '</span>' in get_missing_smi_from_drugBank[i]:
        get_missing_smi_from_drugBank[i] = 'garbage'
        dr_check_again.append(missing_smi_drug_name[i])

In [501]:
for i in range(len(get_missing_smi_from_drugBank)):
    if get_missing_smi_from_drugBank[i] == 'garbage':
        get_missing_smi_from_drugBank[i] = 'neither_in_drugbank_nor_in_supercyp'

In [503]:
#thiamine_smi = 'CC1=NC(N)=C(C[N+]2=CSC(CCO)=C2C)C=N1'
for i in range(len(missing_smi_drug_name)):
    if missing_smi_drug_name[i] == 'Thiamine (vit B1)':
        get_missing_smi_from_drugBank[i] = 'CC1=NC(N)=C(C[N+]2=CSC(CCO)=C2C)C=N1'

In [505]:
# get_missing_smi_from_drugBank

In [506]:
# missing_smi_drug_name

In [507]:
labels = ['missing_drug_name']
d = pd.DataFrame(missing_smi_drug_name, columns=labels)
d['smi_from_drugbank'] = get_missing_smi_from_drugBank
d.to_csv('../dataset/transformer/smi_from_drugbank.csv')

In [511]:
d_index = d.set_index('missing_drug_name')
for i in tqdm_notebook(range(len(df_deduplicated))):
    try:
        smi = d_index.loc[df_deduplicated['drug_name'][i]]['smi_from_drugbank']
        df_deduplicated['smiles'][i] = smi
    except:
        pass

In [512]:
df_deduplicated[df_deduplicated.drug_name == 'Thiamine (vit B1)']

,drug_name,drug_type,smiles
617,Thiamine (vit B1),Inducer,CC1=NC(N)=C(C[N+]2=CSC(CCO)=C2C)C=N1


In [513]:
df_deduplicated.to_csv('../dataset/transformer/from_drugbank_and_transformer_cleaned_v1.csv')

In [516]:
#below_three_from_pubchem
selenium_smi = '[Se]'
glucagon_smi = 'CC(C)CC(C(=O)NC(CCSC)C(=O)NC(CC(=O)N)C(=O)NC(C(C)O)C(=O)O)NC(=O)C(CC1=CNC2=CC=CC=C21)NC(=O)C(CCC(=O)N)NC(=O)C(C(C)C)NC(=O)C(CC3=CC=CC=C3)NC(=O)C(CC(=O)O)NC(=O)C(CCC(=O)N)NC(=O)C(C)NC(=O)C(CCCNC(=N)N)NC(=O)C(CCCNC(=N)N)NC(=O)C(CO)NC(=O)C(CC(=O)O)NC(=O)C(CC(C)C)NC(=O)C(CC4=CC=C(C=C4)O)NC(=O)C(CCCCN)NC(=O)C(CO)NC(=O)C(CC5=CC=C(C=C5)O)NC(=O)C(CC(=O)O)NC(=O)C(CO)NC(=O)C(C(C)O)NC(=O)C(CC6=CC=CC=C6)NC(=O)C(C(C)O)NC(=O)CNC(=O)C(CCC(=O)N)NC(=O)C(CO)NC(=O)C(CC7=CN=CN7)N'
bemiparin_smi = 'C1=CC=C(C=C1)C=CC(=O)O'
#below_two_from_chembl
teicoplanin_smi = 'CC(=O)N[C@H]1[C@@H](O)[C@H](O)[C@H](CO)O[C@H]1O[C@H]2[C@@H]3NC(=O)[C@H](NC(=O)[C@@H]4NC(=O)[C@H]5NC(=O)[C@@H](Cc6ccc(Oc7cc4cc(Oc8ccc2cc8Cl)c7O[C@@H]9O[C@@H](CO)[C@@H](O)[C@H](O)[C@@H]9NC(=O)C)c(Cl)c6)NC(=O)[C@H](N)c%10ccc(O)c(Oc%11cc(O)cc5c%11)c%10)c%12ccc(O)c(c%12)c%13c(O[C@H]%14O[C@H](CO)[C@@H](O)[C@@H](O)[C@@H]%14O)cc(O)cc%13[C@H](NC3=O)C(=O)O'
acriflavimium_chloride_smi = 'Cl.[Cl-].C[n+]1c2cc(N)ccc2cc3ccc(N)cc13.Nc4ccc5cc6ccc(N)cc6nc5c4'

In [517]:
c = 0
for i in tqdm_notebook(range(len(df_deduplicated))):
    
    if df_deduplicated['drug_name'][i] == 'Selenium Compounds':
        df_deduplicated['smiles'][i] = selenium_smi
        continue
        
    elif df_deduplicated['drug_name'][i] == 'Glucagon':
        df_deduplicated['smiles'][i] = glucagon_smi
        continue
        
    elif df_deduplicated['drug_name'][i] == 'Bemiparin':
        df_deduplicated['smiles'][i] = bemiparin_smi
        continue
        
    elif df_deduplicated['drug_name'][i] == 'Teicoplanin':
        df_deduplicated['smiles'][i] = teicoplanin_smi
        continue
        
    elif df_deduplicated['drug_name'][i] == 'Acriflavinium Chloride':
        df_deduplicated['smiles'][i] = acriflavimium_chloride_smi
        continue
        
    elif df_deduplicated['smiles'][i] == 'check_again':
        c = c + 1
        continue

    else:
        continue

In [518]:
c

2

In [519]:
df_deduplicated.to_csv('../dataset/transformer/from_drugbank_and_chembl_and_transformer_cleaned_v2.csv')